In [1]:
# Import CANVAS_API_KEY from the .env file
from dotenv import load_dotenv
load_dotenv(verbose=True)

import os
API_KEY = os.getenv("CANVAS_API_KEY")
BASE_URL = os.getenv("CANVAS_BASE_URL")


In [3]:
import pandas as pd
import requests
import json
import numpy as np

In [4]:
def my_request(API_URL):
    headers = {'Authorization': 'Bearer '+ API_KEY}
    r = requests.get(BASE_URL+API_URL, headers=headers)
    return r.json()

In [5]:
r = my_request('/api/v1/courses?enrollment_type=teacher')

response= pd.DataFrame.from_dict(r)
course_ids = response[response['name'].str.contains('AI')]['id']


```
id	name
1116	AI-1
1202	AI-2
1394	AI-3
1574	AI-4
1682	AI-5
```

In [6]:
#let's start by using AI-4 id:1574
course_id =course_ids[4]

### Get student summaries

In [7]:
#Get student_summaries
r = my_request('/api/v1/courses/{}/analytics/student_summaries?per_page=200'.format(course_id))

response= pd.DataFrame.from_dict(r)

In [8]:
tardiness_summary = pd.DataFrame(list(response.tardiness_breakdown))
student_summaries = response.merge(tardiness_summary, left_index=True, right_index=True).drop(['tardiness_breakdown'], axis='columns')
student_summaries.head()

,id,page_views,max_page_views,page_views_level,participations,max_participations,participations_level,missing,late,on_time,floating,total
0,80474,1,4654,1,0,26,0,11,0,0,31,42
1,84741,1248,4654,2,11,26,2,5,3,5,29,42
2,85442,1203,4654,2,21,26,3,2,2,9,29,42
3,81411,1034,4654,2,15,26,2,0,0,11,31,42
4,81413,122,4654,1,0,26,0,13,0,0,29,42


### Getting Student History

In [9]:
# Get student history
# let's look at id: 75494
course_ids = '1574'
student_id = '75494'
r = my_request('/api/v1/courses/{}/analytics/users/{}/activity?per_page=200'.format(course_ids,student_id))


In [10]:
r['page_views']

{'2020-08-03T20:00:00Z': 19,
 '2020-08-03T23:00:00Z': 10,
 '2020-08-04T14:00:00Z': 13,
 '2020-08-05T12:00:00Z': 10,
 '2020-08-05T13:00:00Z': 9,
 '2020-08-05T14:00:00Z': 54,
 '2020-08-08T23:00:00Z': 2,
 '2020-08-09T21:00:00Z': 2,
 '2020-08-09T22:00:00Z': 23,
 '2020-08-10T13:00:00Z': 16,
 '2020-08-10T16:00:00Z': 4,
 '2020-08-10T18:00:00Z': 3,
 '2020-08-10T19:00:00Z': 4,
 '2020-08-11T16:00:00Z': 14,
 '2020-08-13T11:00:00Z': 5,
 '2020-08-13T12:00:00Z': 16,
 '2020-08-13T13:00:00Z': 12,
 '2020-08-13T18:00:00Z': 6,
 '2020-08-13T19:00:00Z': 5,
 '2020-08-17T14:00:00Z': 10,
 '2020-08-18T13:00:00Z': 2,
 '2020-08-18T15:00:00Z': 26,
 '2020-08-18T17:00:00Z': 6,
 '2020-08-18T18:00:00Z': 7,
 '2020-08-18T19:00:00Z': 14,
 '2020-08-18T20:00:00Z': 7,
 '2020-08-19T13:00:00Z': 25,
 '2020-08-19T14:00:00Z': 25,
 '2020-08-19T15:00:00Z': 5,
 '2020-08-19T18:00:00Z': 17,
 '2020-08-20T13:00:00Z': 15,
 '2020-08-22T15:00:00Z': 58,
 '2020-08-23T22:00:00Z': 2,
 '2020-08-24T13:00:00Z': 18,
 '2020-08-25T11:00:00Z': 1,
 

In [11]:
pd.DataFrame(list(zip(list(r['page_views']),r['page_views'].values())),columns=['date', 'pageviews']).head()

,date,pageviews
0,2020-08-03T20:00:00Z,19
1,2020-08-03T23:00:00Z,10
2,2020-08-04T14:00:00Z,13
3,2020-08-05T12:00:00Z,10
4,2020-08-05T13:00:00Z,9


In [12]:
pd.DataFrame.from_dict(r['participations']).head()

,created_at,url
0,2020-08-05T14:32:10Z,https://student.emeritus.org/courses/1574/quiz...
1,2020-08-05T14:32:34Z,https://student.emeritus.org/courses/1574/quiz...
2,2020-08-05T14:49:26Z,https://student.emeritus.org/api/v1/courses/15...
3,2020-08-10T18:23:03Z,https://student.emeritus.org/api/v1/courses/15...
4,2020-08-13T12:46:09Z,https://student.emeritus.org/api/v1/courses/15...


### Get student outcomes

In [13]:
#get sections
course_id = 1574
                                      
r = my_request('/api/v1/courses/{}/sections?per_page=200'.format(course_id))

In [14]:
pd.DataFrame.from_dict(r)[['id','name']]

,id,name
0,4341,Berkeley Haas Artificial Intelligence
1,4606,Deferment
2,4607,Observers
3,4608,Section A
4,4609,Section B
5,4610,Section C
6,4604,Support
7,4605,Withdrawal


In [159]:
#get assignment id (contains all sections)
course_id = '1574'
                                      
r = my_request('/api/v1/courses/{}/assignments?per_page=200'.format(course_id))

In [160]:
all_assignments = pd.DataFrame(r)
all_assignments['name']

0      Discussion 3.1 - Penalizing Mistakes - Section A
1     Assignment 3.1 - Neural Networks – Teachable M...
2        Assignment 4.1: Vision Recognition - Section A
3     Assignment 4.2: Advanced Language Models - Sec...
4     Assignment 8.1 - Submit Your Capstone Assignme...
                            ...                        
89             Week 6: Q&A Discussion Board - Section C
90             Week 7: Q&A Discussion Board - Section B
91             Week 8: Q&A Discussion Board - Section B
92             Week 8: Q&A Discussion Board - Section C
93             Week 7: Q&A Discussion Board - Section C
Name: name, Length: 94, dtype: object

In [166]:
assignments = all_assignments[all_assignments.name.str.contains('Assignment')
                             ^all_assignments.name.str.contains('Capstone')]
# capstones = all_assignments[capstone_positions]
# assignments = assignments[['id','name']]
# capstones = capstones[['id','name']]
assignments['name']
#capstones

1     Assignment 3.1 - Neural Networks – Teachable M...
2        Assignment 4.1: Vision Recognition - Section A
3     Assignment 4.2: Advanced Language Models - Sec...
7     Assignment 1.1 - Opportunities for AI - Section A
10    Assignment 2.1 - Strategies for Descriptive, P...
11    Assignment 5.1 - Reinforcement Learning: MDPs ...
12    Assignment 5.2 - Robotics Applications - Secti...
15             Assignment 6.1 - AI Strategy - Section A
20    Assignment 7.1 - AI and Your Organization - Se...
28    Assignment 1.1 - Opportunities for AI - Section B
29    Assignment 1.1 - Opportunities for AI - Section C
34    Assignment 2.1 - Strategies for Descriptive, P...
35    Assignment 2.1 - Strategies for Descriptive, P...
38    Assignment 3.1 - Neural Networks – Teachable M...
39    Assignment 3.1 - Neural Networks – Teachable M...
42      Assignment 4.1: Vision Recognition  - Section B
44      Assignment 4.1: Vision Recognition  - Section C
46    Assignment 4.2: Advanced Language Models -

In [123]:
#get list of assignments per student
course_id = '1574'
student_id = '71164'

r = my_request('/api/v1/courses/{}/students/submissions?student_ids[]={}&per_page=200'.format(course_id,student_id))

In [153]:
pd.DataFrame.from_dict(r)[['grade','assignment_id','late','missing','seconds_late']]

,grade,assignment_id,late,missing,seconds_late
0,complete,63387,False,False,0
1,complete,63388,False,False,0
2,complete,63389,False,False,0
3,complete,63390,False,False,0
4,complete,63391,False,False,0
5,complete,63392,False,False,0
6,complete,63393,False,False,0
7,complete,63396,True,False,1205
8,None,63676,False,True,8676103
9,None,63679,False,True,7466503


In [140]:
student_submissions = pd.DataFrame.from_dict(r)[['grade','assignment_id','late','missing','seconds_late']]
student_submissions['grade'].replace(to_replace='complete',value=1, inplace=True)
student_submissions['grade'].fillna(0,inplace=True)
student_submissions['grade'] = student_submissions['grade'].astype(int) #convert boolean to int
student_submissions['late'] = student_submissions['late'].astype(int)
student_submissions['missing'] = student_submissions['missing'].astype(int)
student_submissions

,grade,assignment_id,late,missing,seconds_late
0,1,63387,0,0,0
1,1,63388,0,0,0
2,1,63389,0,0,0
3,1,63390,0,0,0
4,1,63391,0,0,0
5,1,63392,0,0,0
6,1,63393,0,0,0
7,1,63396,1,0,1205
8,0,63676,0,1,8676103
9,0,63679,0,1,7466503


In [142]:
# calculate pass fail
pd.merge(student_submissions, valid_assignments, left_on='assignment_id', right_on='id', how='inner')

,grade,assignment_id,late,missing,seconds_late,id,name
0,1,63389,0,0,0,63389,"Assignment 2.1 - Strategies for Descriptive, P..."
1,1,63391,0,0,0,63391,Assignment 1.1 - Opportunities for AI - Section A
2,1,63393,0,0,0,63393,Assignment 3.1 - Neural Networks – Teachable M...
3,1,65592,1,0,172093,65592,Assignment 7.1 - AI and Your Organization - Se...
4,1,65595,1,0,165194,65595,Assignment 6.1 - AI Strategy - Section A
5,1,65596,1,0,216343,65596,Assignment 5.2 - Robotics Applications - Secti...
6,1,65597,1,0,213866,65597,Assignment 5.1 - Reinforcement Learning: MDPs ...
7,1,65598,1,0,169379,65598,Assignment 4.1: Vision Recognition - Section A
8,1,65599,1,0,173941,65599,Assignment 4.2: Advanced Language Models - Sec...
9,1,65600,0,0,0,65600,Assignment 8.1 - Submit Your Capstone Assignme...


In [21]:
# get capstone project success rate assignment_id=65600
course_id = 1574
assignment_id = 65600 #to-do get list of assignments
r = my_request('/api/v1/courses/{}/assignments/{}/submissions'.format(course_id,assignment_id))

In [22]:
response= pd.DataFrame.from_dict(r)
response[response['id']==63393]


,id,body,url,grade,score,submitted_at,assignment_id,user_id,submission_type,workflow_state,...,grading_period_id,extra_attempts,posted_at,late,missing,seconds_late,entered_grade,entered_score,preview_url,attachments


In [23]:
response[response['id']==63373]

,id,body,url,grade,score,submitted_at,assignment_id,user_id,submission_type,workflow_state,...,grading_period_id,extra_attempts,posted_at,late,missing,seconds_late,entered_grade,entered_score,preview_url,attachments


In [24]:
response[response['id']==63374]

,id,body,url,grade,score,submitted_at,assignment_id,user_id,submission_type,workflow_state,...,grading_period_id,extra_attempts,posted_at,late,missing,seconds_late,entered_grade,entered_score,preview_url,attachments


In [25]:
response['assignment_group_id'].unique()

KeyError: 'assignment_group_id'